# Analytical computation of the Maximum formation efficiency

As described in section 2 from the paper


In [9]:
import numpy as np
import os 
import sys
import pandas as pd
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import matplotlib.lines as mlines
import multiprocessing as mp

# add run_data path to sys
sys.path.append('./run_data')
from definitions import sim_flags_dict


######################################
## PLOT setttings
plt.rc('font', family='serif')
from matplotlib import rc
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
fsize, SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 30,20,25,30
for obj in ['axes','xtick','ytick']:
    plt.rc(obj, labelsize=SMALL_SIZE)          # controls default text sizes
for obj in ['figure','axes']:
    plt.rc(obj, titlesize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize



home_dir    = os.path.expanduser("~") 
compas_v    = "v03.01.02" #"v02.46.01" # #"#v02.35.02/"
datar_root  =  f"{home_dir}/ceph/CompasOutput/{compas_v}/"


## Approximate maximum formation efficiency

We approximate the 'maximum' formation efficiency using a drake equation:

\begin{equation}
     \eta_{form} = \frac{1}{\langle M_{SF} \rangle } \Bigl( f_{\mathrm{primary}} \times f_{\mathrm{secondary}} \times f_{\mathrm{init \, sep}} \times f_{\mathrm{survive \, SN1}} \times f_{\mathrm{survive \, SN2}} \Bigr)
\end{equation}


### Probability of forming 

We approximate the terms _within the round brackets of Equation 2 above_ with the probability to form a pair of massive stars with the `right' set of (initial) conditions. The conditions in questions are random variables at ZAMS (the primary mass, $m_1$, secondary mass, $m_2$, orbital separation, $a$), and factors affecting the stars' survival during supernovae in the first and second mass transfer phases. 

\begin{equation}
\begin{split}
p(m_1, ~m_2, ~a, ~\text{survive SN1}, ~\text{survive SN2}) 
& = p(m_1) \times p(m_2~|~m_1) \\
& \times p(a ~|~m_1, ~m_2) \\
& \times  p(\text{survive SN1}~|~m_1, ~m_2, ~a) \\
& \times p(\text{survive SN2}~|~m_1, ~m_2, ~a, ~\text{survive SN1})
\end{split}
\end{equation}

By integrating over the relevant ranges of $m_1, ~q, ~a, ~\text{survive SN1}, ~\text{survive SN2} \in C$, we obtain the probability for a specific type of compact binary merger to occur. 


##  Primary and secondary masses
We compute the probability that $m_2$ falls within a certain mass range [c,d] given that $m_1$ within a certain mass range [a,b]:

\begin{equation} 
\begin{split}
f_{\text{primary}} \times f_{\text{secondary}} 
&  \approx   \int_{m_1 = a}^{m_1 = b} \int_{m_2 = c}^{m_2 = d} p(m_1) \times p(m_2 | m_1)
\end{split}
\end{equation}

We can write $m_2$ in terms of the mass ratio $q \equiv m_2/m_1 < 1$:
\begin{equation}
    p(m_2~|~m_1) = \frac{\partial q}{\partial m_2} p(q~|~m_1) = \frac{1}{m_1} p(q~|~m_1)
\end{equation}

We assume $p(q)$ follows a uniform probability distribution between $0$ and $1$:
\begin{equation} 
    p(q~|~m_1) = U(q|0,1)
\end{equation} 


We assume $p(m_1)$ follows the Kroupa IMF, defined as:
\begin{equation} 
\text{Kroupa IMF}(m_1) = C_1
    \begin{cases}
      (m_1/M_{\odot})^{-0.3} & 0.01~M_{\odot} \leq m_1 < 0.08~M_{\odot}\\
      C_2 \cdot (m_1/M_{\odot})^{-1.3} & 0.08~M_{\odot} \leq m_1 < 0.5~M_{\odot}\\
      C_3 \cdot (m_1/M_{\odot})^{-2.3} & 0.5~M_{\odot} \leq m_1 < 300~M_{\odot}\\
      0 & \text{otherwise}
    \end{cases}       
\end{equation}

Here $C_2 = 0.08^{-0.3 + 1.3} = 0.08$ and $C_3 = C_2 \cdot 0.5^{-1.3 + 2.3} = C_2 \cdot 0.5$ to ensure continuity.
C_1 is a normalization constant, determined by normalizing the IMF over the entire mass range:
\begin{equation*}
    C_1 = \Big[ \frac{0.08^{(-0.3+1)}-0.01^{(-0.3+1)}}{(-0.3+1)} + 0.08 \cdot \left( \frac{0.5^{(-1.3+1)}-0.08^{(-1.3+1)}}{(-1.3+1)} \right) + 0.08 \cdot 0.5 \cdot \left(\frac{300^{(-2.3+1)}-0.5^{(-2.3+1)}}{(-2.3+1)} \right)  \Big]^{-1}
\end{equation*}
Note that we set the minimum and maximum stellar masses to $0.01~M_{\odot}$ and $300~M_{\odot}$, respectively.



given a fixed primary mass $m_1$, $p(q~|~m_1)$ represents the probability density of finding a partner star such that the mass ratio becomes $q$. 



Combining equations above we get:

\begin{equation}
\begin{split}
        f_{\text{primary}} \times f_{\text{secondary}} 
        & = \int_{m_1=a}^{m_1=b} dm_1 \int_{q=c/m_1}^{q=d/m_1} dq ~\text{Kroupa IMF}(m_1) \times \frac{M_{\odot}}{m_1}U(q|0,1) \\
        & = \int_{m_1=a}^{m_1=b} dm_1 ~\text{Kroupa IMF}(m_1) \times \frac{1}{m_1} \left( \frac{d}{m_1} - \frac{c}{m_1} \right)  \\
        & = (d -c) \int_{m_1=a}^{m_1=b} dm_1 ~\text{Kroupa IMF}(m_1) \times \frac{1}{m_1^2} \\
\end{split}
\end{equation}

In [30]:
# the integral of a piece (or term) of a broken powerlaw
def norm_term(minm, maxm, power):
    return (maxm**(power + 1) - minm**(power + 1)) / (power + 1)

term1 = norm_term(0.01, 0.08, -0.3)
term2 = 0.08 * norm_term(0.08, 0.5, -1.3)
term3 = 0.08 * 0.5 * norm_term(0.5, 300, -2.3)

Norm = 1/(term1+ term2 + term3 )
print(Norm)


1.9869192439784182


### BHBH

For BHBH, we assume both primary and secondary masses range from [$20M_{\odot}$,$300M_{\odot}$]

\begin{equation}
\begin{split}
    f_{\text{primary}} \times f_{\text{secondary}}
    & = C_1  \left(300 - 20 \right)\int_{m_1=20M_{\odot}}^{m_1=300M_{\odot}} dm_1 C_3 \cdot m_1^{-2.3} \times m_1^{-2} \\
    & = C_1  \left(300 - 20 \right) C_3\int_{m_1=20M_{\odot}}^{m_1=300M_{\odot}} dm_1  \cdot m_1^{-4.3} \\
    & = C_1  \left(300 - 20 \right)  \frac{C_3 }{-3.3} \left| m_1^{-3.3} \right|^{300}_{20} \\
    & = C_1  \left(300 - 20 \right)  \frac{C_3 }{-3.3} \left( 300^{-3.3} - 20^{-3.3} \right) \\
    & = 3.4 \times 10^{-4}
\end{split}
\end{equation}
<!-- 2.43 \times 10^{-4} -->




<!--     & = C_1 \int_{m_1=20M_{\odot}}^{m_1=300M_{\odot}} dm_1 \int_{q=20/m_1}^{q=300/m_1} dq ~\Big(\frac{m_1}{1}\Big)^{-2.7} \times \frac{1}{m_1} \\
    & = C_1 \int_{m_1=20M_{\odot}}^{m_1=300M_{\odot}} \left(\frac{300}{m_1} - \frac{20}{m_1} \right) ~\Big(\frac{m_1}{1}\Big)^{-3.7} \\
    & = C_1 \int_{m_1=20M_{\odot}}^{m_1=300M_{\odot}} m_1 ^{-1} \left(300 - 20 \right) ~\Big(\frac{m_1}{1}\Big)^{-3.7} \\
    & = C_1  \left(300 - 20 \right) \int_{m_1=20M_{\odot}}^{m_1=300M_{\odot}} ~\Big(\frac{m_1}{1}\Big)^{-4.7} \\
    & = C_1  \frac{\left(300 - 20 \right)}{-3.7} \Big[ \left(\frac{m_1}{1}\right)^{-3.7} \Big]^{300}_{20} \\
    & = C_1  \frac{\left(300 - 20 \right)}{-3.7} \Big[ 300^{-3.7} - 20^{-3.7} \Big]\\ -->

As long as our masses $m_1 >0.5 M_{\odot}$, we can write this more generally as: 
\begin{equation}
    f_{\text{primary}} \times f_{\text{secondary}} = C_1  \frac{(d - c) C_3 }{-3.3} \Big[b^{-3.3} - a^{-3.3} \Big]
\end{equation}

for $m_1 = [a,b]$ and $m_2 = [c,d]$

In [56]:
C3 = 0.08 * 0.5
print( (Norm * (300 - 20) * C3) /-3.3 * (300**-3.3 - 20**-3.3) ) 

def f_primaryXf_secondary(power, a,b,c,d, C1 = Norm, C3 = (0.08*0.5)):
    # here power is the powerlaw that the mass distribution pdf follows. 
    # This only works if a-b, and c-d all fall within one leg of the Kroupa IMF
    # i.e. have the same power alpha
    # C_3 = 0.08 * 0.5    # = C2 * 0.5^(-1.3 + 2.3), with C_2 = 0.08^(-0.3 + 1.3)
    return  (C1 * (d-c)* C3 ) / (power- 1) * (b**(power-1) - a**(power-1))

f_primaryf_secondary_bhbh = f_primaryXf_secondary(-2.3, 20,300,20,300, C1 = Norm, C3 = (0.08*0.5)) 
print(f"{f_primaryf_secondary_bhbh} = {f_primaryf_secondary_bhbh:.2E}")


0.0003431059134688049
0.0003431059134688049 = 3.43E-04


### NSNS

For NSNS we assume both primary and secondary masses range from [$8M_{\odot}$,$20M_{\odot}$]

<!-- \begin{equation}
\begin{split}
    f_{\text{primary}} \times f_{\text{secondary}}
    & = (\text{constant}) \int_{m_1=8M_{\odot}}^{m_1=20M_{\odot}} d(m_1/M_{\odot}) \int_{q=8/m_1}^{q=20/m_1} dq ~\Big(\frac{m_1}{M_{\odot}}\Big)^{-2.7} \times \frac{M_{\odot}}{m_1} \\
    & = \text{constant} \int_{m_1=8M_{\odot}}^{m_1=20M_{\odot}} \left(\frac{20}{m_1} - \frac{8}{m_1} \right) ~\Big(\frac{m_1}{M_{\odot}}\Big)^{-3.7} \\
    & = \text{constant} \int_{m_1=8M_{\odot}}^{m_1=20M_{\odot}} m_1 ^{-1} \left(20 - 8 \right) ~\Big(\frac{m_1}{M_{\odot}}\Big)^{-3.7} \\
    & = \text{constant}  \left(20 - 8 \right) \int_{m_1=8M_{\odot}}^{m_1=20M_{\odot}} ~\Big(\frac{m_1}{M_{\odot}}\Big)^{-4.7} \\
    & = \text{constant}  \frac{\left(20 - 8 \right)}{-3.7} \Big[ \left(\frac{m_1}{M_{\odot}}\right)^{-3.7} \Big]^{20}_{8} \\
    & = \text{constant}  \frac{\left(20 - 8 \right)}{-3.7} \Big[ 20^{-3.7} - 8^{-3.7} \Big]\\
    & = 2.43 \times 10^{-4}
\end{split}
\end{equation} -->

In [60]:
print( ((Norm * (20 - 8) * C3 ) /-3.7) * (20**-3.7 - 8**-3.7) ) 

f_primaryf_secondary_nsns = f_primaryXf_secondary(-2.7, 8,20,8,20, C1 = Norm, C3 = (0.08*0.5)) 

print(f"{f_primaryf_secondary_nsns} = {f_primaryf_secondary_nsns:.2E}" )


0.0001134746975095127
0.0001134746975095127 = 1.13E-04


### BHNS

Lastly for BHNS we assume $m_1 = [20M_{\odot}$,$300M_{\odot}]$ while $m_2 = [8M_{\odot}$,$20M_{\odot}]$


In [61]:
print( ((Norm * (20 - 8) * C3)/-3.7) * (300**-3.7 - 20**-3.7) ) 

f_primaryf_secondary_bhns =  f_primaryXf_secondary(-2.7, 20,300,8,20, C1 = Norm, C3 = (0.08*0.5))
print(f"{f_primaryf_secondary_bhns} = {f_primaryf_secondary_bhns:.2E}" )


3.957212883472308e-06
3.957212883472308e-06 = 3.96E-06


## Initial separation

for $f_{init sep}$, we adopt the fraction of systems that interacts ever. 

We assume that binaries can form with separations between 0.01AU and 1000 AU 

we assume a flat-in-log distribution of initial separations

\begin{equation}
P(a_i) = 1/a_i
\end{equation}

\begin{equation}
f_{init sep} = \frac{\log(a_i)|_{mina}^{maxa} }{\log(a_i)|_{0.01 AU}^{1000 AU} }
\end{equation}

where $mina$ and $maxa$ are the min and max separation for interaction,
For the minimum, we look at our case A,B C plots, and stars have radii of $3-20R_{\odot}$ at birth (roughly), leading to a range of 0.0279AU - 0.186AU. We adopt the average of $mina \approx 0.1 AU$

For the upper end, we use our max R per Z to estimate this. Very roughly stars range between a max R of 1000 and 5000, so we adopt $maxa \approx 3000 R_{\odot} = 13.95 AU$


\begin{equation}
f_{init sep} = \frac{\log(13.95) - \log(0.1)}{\log(1000) - \log(0.01)} \approx 0.43
\end{equation}


In [62]:
f_init_sep = (np.log(14) - np.log(0.1))/ (np.log(1000) - np.log(0.01))
print( np.round(f_init_sep,2) )

0.43


## Probability to survive SN1 and SN2

 We assume no kicks for BBH so $f_{SN1} = f_{SN2} = 1$

 We assume full kicks for NSNS so $f_{SN2} = f_{SN1} \approx 0.2$ (See derivationin appendix A3)

In [63]:
f_sn1_bbh = 1.
f_sn2_bbh = 1.

f_sn1_bhns = 1.
f_sn2_bhns = 1. #0.2

f_sn1_nsns = 0.2
f_sn2_nsns = 1.0



## Average SF mass per binary system

We use the 'total mass evolved per Z' function that we also use for the yield calculations for this

In [67]:
def get_totalMassInStarFormation(x2=0.08, x3=0.5, a1=-0.3, a2=-1.3, a3=-2.3, C1=1.,
                         binaryFraction=0.7, Mmin_universe=0.01, Mmax_universe=300., sampleSize=2000000):
    """_summary_

    Args:
        # COMPAS simulation parameters
        pathCOMPASh5 (_type_, optional): path to your COMPAS file. Defaults to None.

        # Broken powerlaw (Kroupa IMF) parameters
        x1, x2, x3, x4: float, the break points (mass ranges) for the three segments
        a1, a2, a3: float, the power law indices 
        <0.01 - 0.08> a = -0.3, <0.08 - 0.5> a = -1.3, <0.5 - 200> a = -2.3
        C1: float, the normalization constant for the first segment
        
        # Believes about star formation in the Universe
        binaryFraction (int, optional): What fraction of stars are in binaries. Default= 1.
        Mmin_universe, Mmax_universe (float): the min and max mass that stars in the Universe can be born with  Defaults: 0.01 and 200.

    Returns:
        _type_: _description_
    """ 
    x1 = Mmin_universe
    x4 = Mmax_universe

    ##########################
    # Create Sample Universe 
    ##########################
    # we will use 'inverse transform sampling method' to sample our sample Universe from the IMF

    ### Primary mass
    # first we compute the y-values of the CDF of our IMF at Mmin_universe and Mmax_universe
    # Mmin_universe and Mmax_universe have to be between x1 and x4
    CDFmin = CDFbrokenPowerLaw(np.array([Mmin_universe]), x1, x2, x3, x4, a1, a2, a3, C1)
    CDFmax = CDFbrokenPowerLaw(np.array([Mmax_universe]), x1, x2, x3, x4, a1, a2, a3, C1)

    # Now we can sample Uniformly from the CDF between CDFmin and CDFmax
    drawM1      = np.random.uniform(CDFmin,CDFmax,sampleSize)
    # Convert CDF values back to masses
    M1          = invertCDFbrokenPowerLaw(drawM1, x1, x2, x3, x4, a1, a2, a3, C1)

    ### Secondary mass
    # mass ratio (q = m2/m1) distribution is assumed to be flat 
    # so then the drawM2 (if it is in a binary) just becomes the mass fraction.
    drawM2          = np.random.uniform(0,1,sampleSize)    # we are actually sampling q
    M2              = np.zeros(sampleSize)                 # are zeros, but will be filled with binary fraction

    ### Binary fraction
    # we want that binaryFraction of the stars are in binaries
    # Hence by drawing between 0-1, we have to throw out everything that is above binaryFraction (i.e. = single and m2 = 0)
    # to incorporate the mass dependence of f_binary, we bin our samples in mass and draw a binary fraction for each bin
    binary_bin_edges    = [x1, 0.08, 0.5, 1, 10, x4]
    binaryFractions     = [0.1, 0.25, 0.5, 0.75, 1]

    for m_i in range(len(binary_bin_edges[:-1])):
        m1_mask = (M1 >= binary_bin_edges[m_i]) & (M1 < binary_bin_edges[m_i+1])
        drawBinary = np.random.uniform(0,1,np.sum(m1_mask)) # draw a binary for all the samples in this mass bin
        maskBinary = drawBinary < binaryFractions[m_i]
        
        # if maskBinary is True, then M2 = q * m1, else M2 = 0
        M2[m1_mask] = np.where(maskBinary, drawM2[m1_mask] * M1[m1_mask], 0)

    totalMassInStarFormation = np.sum(M1) + np.sum(M2)

    return totalMassInStarFormation


def CDFbrokenPowerLaw(x, x1=0.01, x2=0.08, x3=0.5, x4=200, a1=-0.3, a2=-1.3, a3=-2.3, C1=1):
    """
    CDF values of a three-part broken powerlaw representing a Kroupa IMF by default.
    
    Parameters:
    x: array-like, the input values
    x1, x2, x3, x4: float, the break points (mass ranges) for the three segments
    a1, a2, a3: float, the power law indices 
    C1: float, the normalization constant for the first segment
    
    Returns:
    yvalues: array-like, the output values of the CDF
    """
    
    # Initialize the output array
    yvalues = np.zeros(len(x))
    
    # Calculate the normalization constants for the other segments
    # Ensuring that the next segments start where the previous segment ends
    C2 = float(C1 * (x2**(a1-a2)))
    C3 = float(C2 * (x3**(a2-a3)))
    
    # Calculate the normalization factors for the three segments
    N1 = float(((1./(a1+1)) * C1 * (x2**(a1+1))) - ((1./(a1+1)) * C1 * (x1**(a1+1))))
    N2 = float(((1./(a2+1)) * C2 * (x3**(a2+1))) - ((1./(a2+1)) * C2 * (x2**(a2+1))))
    N3 = float(((1./(a3+1)) * C3 * (x4**(a3+1))) - ((1./(a3+1)) * C3 * (x3**(a3+1))))
    
    # Calculate the denominator of the CDF
    bottom = N1+N2+N3
    
    # Calculate the CDF values for x range: x1<=x<x2
    mask1 = (x>=x1) & (x<x2)
    top1 = ( (1./(a1+1) ) * C1 * (x[mask1]**(a1+1) ) - (1./(a1+1) ) * C1 * (x1**(a1+1) ) ) 
    yvalues[mask1] = top1/bottom
    
    # Calculate the CDF values for x range: x2<=x<x3
    mask2 = (x>=x2) & (x<x3)
    top2 =  N1 + ( (1./(a2+1) ) * C2 * (x[mask2]**(a2+1) ) - (1./(a2+1)) * C2 * (x2**(a2+1) ) ) 
    yvalues[mask2] = top2/bottom
    
    # Calculate the CDF values for x range: x3<=x<=x4
    mask3 = (x>=x3) & (x<=x4)
    top3 =  N1 + N2 + ( (1./(a3+1)) * C3 * (x[mask3]**(a3+1)) - (1./(a3+1)) * C3 * (x3**(a3+1) ) )
    yvalues[mask3] = top3/bottom
    
    return yvalues


def invertCDFbrokenPowerLaw(CDF, x1, x2, x3, x4, a1, a2, a3, C1):
    """
    Invert y-values of a CDF back to x-vals (i.e. the masses)
    Specifically for a three-part piece-wise powerlaw representing a Kroupa IMF by default. 

    Parameters:
    CDF: array-like, the CDF values to invert
    x1, x2, x3, x4: float, the break points (ranges) for the three segments
    a1, a2, a3: float, the power law indices for the three segments
    C1: float, the normalization constant for the first segment

    Returns:
    xvalues: array-like, the inverted CDF values
    """
    
    # Calculate the normalization constants for the second and third segments
    C2 = float(C1 * (x2**(a1-a2)))
    C3 = float(C2 * (x3**(a2-a3)))
    
    # Calculate the area under the curve for each segment
    N1 = float(((1./(a1+1)) * C1 * (x2**(a1+1))) - ((1./(a1+1)) * C1 * (x1**(a1+1))))
    N2 = float(((1./(a2+1)) * C2 * (x3**(a2+1))) - ((1./(a2+1)) * C2 * (x2**(a2+1))))
    N3 = float(((1./(a3+1)) * C3 * (x4**(a3+1))) - ((1./(a3+1)) * C3 * (x3**(a3+1))))
    
    # Calculate the CDF values at the breakpoints
    CDFx2 = CDFbrokenPowerLaw(np.array([x2,x2]), x1, x2, x3, x4, a1, a2, a3, C1)[0]
    CDFx3 = CDFbrokenPowerLaw(np.array([x3,x3]), x1, x2, x3, x4, a1, a2, a3, C1)[0]

    # Initialize the output array
    xvalues = np.zeros(len(CDF))
    
    # Calculate the inverse CDF values for the first segment
    mask1 = (CDF < CDFx2)
    xvalues[mask1] =  (((CDF[mask1]*(N1+N2+N3))  + \
                      ( (1./(a1+1))*C1*(x1**(a1+1))))/((1./(a1+1))*C1))**(1./(a1+1))
    
    # Calculate the inverse CDF values for the second segment
    mask2 = (CDFx2<= CDF) & (CDF < CDFx3)
    xvalues[mask2] = ((((CDF[mask2]*(N1+N2+N3))-(N1))  + \
                      ( (1./(a2+1))*C2*(x2**(a2+1))))/((1./(a2+1))*C2))**(1./(a2+1))
    
    # Calculate the inverse CDF values for the third segment
    mask3 = (CDFx3<= CDF) 
    xvalues[mask3] = ((((CDF[mask3]*(N1+N2+N3))-(N1+N2))  + \
                      ((1./(a3+1))*C3*(x3**(a3+1))))/((1./(a3+1))*C3))**(1./(a3+1))
    
    # Return the inverse CDF values
    return xvalues

In [68]:
Sampe_size = int(5e6)
totalMassInStarFormation =  get_totalMassInStarFormation(sampleSize=Sampe_size, binaryFraction=0.7)

average_mass_per_system_univ = totalMassInStarFormation/Sampe_size
print(f'Average mass per system in Universe {average_mass_per_system_univ}' )

Average mass per system in Universe 0.5145862473992391


*** 
*** 
## max $\eta_{BBH}$

***
***

In [72]:
eta_BBH = 1./average_mass_per_system_univ * (f_primaryf_secondary_bhbh * f_init_sep * f_sn1_bbh * f_sn2_bbh )
print(f'eta_BBH = 1./{np.round(average_mass_per_system_univ,2)} * ({f_primaryf_secondary_bhbh} * {f_init_sep} * {f_sn1_bbh} * {f_sn2_bbh} ) =  {eta_BBH} = {eta_BBH:.2E}' )


eta_BBH = 1./0.51 * (0.0003431059134688049 * 0.4292256071356476 * 1.0 * 1.0 ) =  0.0002861907887449239 = 2.86E-04


***
***

## max $\eta_{BHNS}$

***
***

In [73]:
eta_BHNS = 1./average_mass_per_system_univ * (f_primaryf_secondary_bhns * f_init_sep * f_sn1_bhns * f_sn2_bhns )
print(f'eta_BHNS = 1./{np.round(average_mass_per_system_univ,2)} * ({f_primaryf_secondary_bhns} * {f_init_sep} * {f_sn1_bhns} * {f_sn2_bhns}) =  {eta_BHNS} = {eta_BHNS:.2E}' )

eta_BHNS = 1./0.51 * (3.957212883472308e-06 * 0.4292256071356476 * 1.0 * 1.0) =  3.300782154707697e-06 = 3.30E-06


***
***
## max $\eta_{NSNS}$
***
***

In [74]:
f_sn2_nsns = 1.

eta_NSNS = 1./average_mass_per_system_univ * (f_primaryf_secondary_nsns * f_init_sep * f_sn1_nsns * f_sn2_nsns )
print(f'eta_nsns = 1./{np.round(average_mass_per_system_univ,2)} * ({f_primaryf_secondary_nsns} * {f_init_sep} * {f_sn1_nsns} * {f_sn2_nsns} ) =  {eta_NSNS} = {eta_NSNS:.2E}' )

eta_nsns = 1./0.51 * (0.0001134746975095127 * 0.4292256071356476 * 0.2 * 1.0 ) =  1.8930255590474834e-05 = 1.89E-05
